# 🚗 Safespace Model Testing Notebook

This notebook allows you to test the accident detection model with:
- **Video files** - Test with pre-recorded footage
- **Camera feed** - Real-time detection from webcam

## Setup

In [1]:
# Import Required Libraries
import cv2
import torch
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import supervision as sv
from IPython.display import display, clear_output
import ipywidgets as widgets
import time

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Using device: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

🖥️ Using device: cpu


## Load the Model

In [2]:
# Configuration
MODEL_PATH = "../Models/Car Accident.pt"  # Path to your model
CONFIDENCE_THRESHOLD = 0.5  # Detection confidence threshold

# Load the model
model = YOLO(MODEL_PATH)
model.to(device)
print(f"✅ Model loaded successfully from: {MODEL_PATH}")
print(f"   Classes: {model.names}")

✅ Model loaded successfully from: ../Models/Car Accident.pt
   Classes: {0: 'accident', 1: 'person', 2: 'vehicle'}


## Detection Helper Functions

In [3]:
def detect_and_annotate(frame, model, confidence=0.5):
    """
    Run detection on a frame and return annotated frame with detections.
    
    Args:
        frame: Input BGR frame
        model: YOLO model
        confidence: Detection confidence threshold
        
    Returns:
        annotated_frame: Frame with bounding boxes
        detections: Supervision Detections object
        inference_time: Time taken for inference in ms
    """
    start_time = time.time()
    
    # Run inference
    results = model.predict(frame, conf=confidence, device=device, verbose=False)
    
    inference_time = (time.time() - start_time) * 1000  # Convert to ms
    
    # Convert to supervision detections
    detections = sv.Detections.from_ultralytics(results[0])
    
    # Create annotators
    box_annotator = sv.BoxAnnotator(
        thickness=2,
    )
    label_annotator = sv.LabelAnnotator(
        text_scale=0.5,
        text_thickness=1,
    )
    
    # Generate labels
    labels = [
        f"{model.names[class_id]} {confidence:.2f}"
        for class_id, confidence in zip(detections.class_id, detections.confidence)
    ] if len(detections) > 0 else []
    
    # Annotate frame
    annotated_frame = box_annotator.annotate(scene=frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    
    return annotated_frame, detections, inference_time


def add_stats_overlay(frame, fps, inference_time, detection_count):
    """Add statistics overlay to frame."""
    overlay = frame.copy()
    
    # Add semi-transparent background
    cv2.rectangle(overlay, (10, 10), (250, 100), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
    
    # Add text
    cv2.putText(frame, f"FPS: {fps:.1f}", (20, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Inference: {inference_time:.1f}ms", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Detections: {detection_count}", (20, 85), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    return frame

print("✅ Helper functions defined")

✅ Helper functions defined


---
## Option 1: Test with Video File 🎬

Run the cell below to test the model on a video file. Press **'q'** to quit.

In [7]:
# ========================================
# 🎬 VIDEO FILE TESTING
# ========================================

# Set your video path here
VIDEO_PATH = "/home/ziadmoh/UNI/Safespace_node/assets/Video Test/test5.mp4"  # Change this to your video path

# Open video
cap = cv2.VideoCapture(VIDEO_PATH)

if not cap.isOpened():
    print(f"❌ Error: Could not open video file: {VIDEO_PATH}")
else:
    print(f"✅ Video loaded: {VIDEO_PATH}")
    print(f"   Resolution: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))}x{int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")
    print(f"   FPS: {cap.get(cv2.CAP_PROP_FPS):.1f}")
    print(f"   Total Frames: {int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}")
    print("\n🎮 Controls: Press 'q' to quit, 'p' to pause/resume, 's' to save frame")
    
    paused = False
    frame_count = 0
    fps_start_time = time.time()
    fps = 0
    
    while True:
        if not paused:
            ret, frame = cap.read()
            if not ret:
                # Loop video
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            
            frame_count += 1
            
            # Run detection
            annotated_frame, detections, inference_time = detect_and_annotate(
                frame, model, CONFIDENCE_THRESHOLD
            )
            
            # Calculate FPS
            if frame_count % 10 == 0:
                fps = 10 / (time.time() - fps_start_time)
                fps_start_time = time.time()
            
            # Add stats overlay
            annotated_frame = add_stats_overlay(
                annotated_frame, fps, inference_time, len(detections)
            )
            
            # Show alert if accident detected
            if len(detections) > 0:
                cv2.putText(annotated_frame, "⚠️ ACCIDENT DETECTED!", 
                           (annotated_frame.shape[1]//2 - 200, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
            
            cv2.imshow("Safespace Model Test - Video", annotated_frame)
        
        # Handle key presses
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('p'):
            paused = not paused
            print("⏸️ Paused" if paused else "▶️ Resumed")
        elif key == ord('s'):
            save_path = f"detection_frame_{frame_count}.jpg"
            cv2.imwrite(save_path, annotated_frame)
            print(f"💾 Frame saved: {save_path}")
    
    cap.release()
    cv2.destroyAllWindows()
    print("\n✅ Video testing completed!")

✅ Video loaded: /home/ziadmoh/UNI/Safespace_node/assets/Video Test/test5.mp4
   Resolution: 360x640
   FPS: 30.0
   Total Frames: 315

🎮 Controls: Press 'q' to quit, 'p' to pause/resume, 's' to save frame
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ziadmoh/Programs/anaconda3/envs/AI_ENV/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_436581/3340360750.py", line 36, in <module>
    annotated_frame, detections, inference_time = detect_and_annotate(
                                                  ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_436581/4254842974.py", line 18, in detect_and_annotate
    results = model.predict(frame, conf=confidence, device=device, verbose=False)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ziadmoh/Programs/anaconda3/envs/AI_ENV/lib/python3.11/site-packages/ultralytics/engine/model.py", line 536, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
                                                                    ^^^^^^

---
## Option 2: Test with Camera Feed 📷

Run the cell below to test the model with your webcam. Press **'q'** to quit.

In [8]:
# ========================================
# 📷 CAMERA FEED TESTING
# ========================================

# Camera settings
CAMERA_INDEX = 0  # Change if you have multiple cameras
CAMERA_WIDTH = 1280
CAMERA_HEIGHT = 720

# Open camera
cap = cv2.VideoCapture(CAMERA_INDEX)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAMERA_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAMERA_HEIGHT)

if not cap.isOpened():
    print(f"❌ Error: Could not open camera (index: {CAMERA_INDEX})")
else:
    actual_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    actual_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"✅ Camera opened successfully")
    print(f"   Resolution: {actual_width}x{actual_height}")
    print("\n🎮 Controls: Press 'q' to quit, 's' to save frame")
    
    frame_count = 0
    fps_start_time = time.time()
    fps = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Failed to grab frame")
            break
        
        frame_count += 1
        
        # Run detection
        annotated_frame, detections, inference_time = detect_and_annotate(
            frame, model, CONFIDENCE_THRESHOLD
        )
        
        # Calculate FPS
        if frame_count % 10 == 0:
            fps = 10 / (time.time() - fps_start_time)
            fps_start_time = time.time()
        
        # Add stats overlay
        annotated_frame = add_stats_overlay(
            annotated_frame, fps, inference_time, len(detections)
        )
        
        # Show alert if accident detected
        if len(detections) > 0:
            cv2.putText(annotated_frame, "⚠️ ACCIDENT DETECTED!", 
                       (annotated_frame.shape[1]//2 - 200, 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        
        cv2.imshow("Safespace Model Test - Camera", annotated_frame)
        
        # Handle key presses
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            save_path = f"camera_detection_{frame_count}.jpg"
            cv2.imwrite(save_path, annotated_frame)
            print(f"💾 Frame saved: {save_path}")
    
    cap.release()
    cv2.destroyAllWindows()
    print("\n✅ Camera testing completed!")

✅ Camera opened successfully
   Resolution: 1280x720

🎮 Controls: Press 'q' to quit, 's' to save frame

✅ Camera testing completed!


---
## Option 3: Test Single Image 🖼️

Test the model on a single image file.

In [ ]:
# ========================================
# 🖼️ SINGLE IMAGE TESTING
# ========================================
import matplotlib.pyplot as plt

# Set your image path here
IMAGE_PATH = "../../safespace/assets/accidents_images/test_accident.jpg"  # Change this

# Check if file exists, if not create a test from video
if not Path(IMAGE_PATH).exists():
    print(f"⚠️ Image not found: {IMAGE_PATH}")
    print("   Extracting a frame from video for testing...")
    
    test_video = "../../safespace/assets/Video Test/test5.mp4"
    cap = cv2.VideoCapture(test_video)
    if cap.isOpened():
        # Skip to middle of video
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / 2))
        ret, frame = cap.read()
        if ret:
            IMAGE_PATH = "test_frame.jpg"
            cv2.imwrite(IMAGE_PATH, frame)
            print(f"   ✅ Test frame saved: {IMAGE_PATH}")
        cap.release()

# Load and process image
if Path(IMAGE_PATH).exists():
    image = cv2.imread(IMAGE_PATH)
    
    print(f"📷 Processing image: {IMAGE_PATH}")
    print(f"   Size: {image.shape[1]}x{image.shape[0]}")
    
    # Run detection
    annotated_image, detections, inference_time = detect_and_annotate(
        image, model, CONFIDENCE_THRESHOLD
    )
    
    # Print results
    print(f"\n📊 Results:")
    print(f"   Inference time: {inference_time:.2f}ms")
    print(f"   Detections: {len(detections)}")
    
    if len(detections) > 0:
        print(f"\n🚨 Detected objects:")
        for i, (class_id, conf) in enumerate(zip(detections.class_id, detections.confidence)):
            print(f"   {i+1}. {model.names[class_id]} - Confidence: {conf:.2%}")
    
    # Display image
    plt.figure(figsize=(14, 8))
    plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
    plt.title(f"Detection Results - {len(detections)} object(s) found")
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print(f"❌ Could not load image: {IMAGE_PATH}")

---
## Cleanup

Run this cell to close any remaining windows.

In [9]:
# Cleanup - close all OpenCV windows
cv2.destroyAllWindows()
print("🧹 All windows closed")

🧹 All windows closed
